In [1]:
import glob, os
import numpy as np

from astrobase import periodbase, checkplot, varbase
from astrobase.periodbase.kbls import bls_snr, bls_parallel_pfind
from astrobase.varbase import lcfit

from astropy.io import ascii
from astropy.table import Table, join, Column
from decimal import getcontext, Decimal
from pylab import *

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
%matplotlib inline

# Spectral Classification Bounds

In [2]:
O_c=np.linspace(-99,-0.17,2,endpoint=True)
B_c=np.linspace(-0.17,-0.01,2,endpoint=True)
A_c=np.linspace(-0.01,0.15,2,endpoint=True)
F_c=np.linspace(0.15,0.3,2,endpoint=True)
G_c=np.linspace(0.3,0.5,2,endpoint=True)
K_c=np.linspace(0.5,0.8,2,endpoint=True)
M_c=np.linspace(0.8,999,2,endpoint=True)

# Searched Variable Classes -- from VSX Definitions & Literature

In [3]:
##Pulsators
typeA='Pulsator: BCephei' #Non-superigiant pulsating O8-B6 stars with light and radial-velocity variations \
#caused by low-order pressure and gravity mode pulsations.
PA=np.linspace(0.1,0.6,2,endpoint=True) 
dmagA=np.linspace(0.01,0.3,2,endpoint=True) 
JmKA=np.linspace(-99,-0.01,2,endpoint=True)

typeB='Pulsator: Dscuti' # Radial AND Non-radial pulsating variables of spectral types A0-F5 III-V
PB=np.linspace(0.02,0.25,2,endpoint=True) 
dmagB=np.linspace(0.00001,0.9,2,endpoint=True) #ignore lower mag limit per Joel
JmKB=np.linspace(-0.01,0.3,2,endpoint=True)

typeC='Pulsator: RRL' # Radially-pulsating giant A-F stars --(horizontal-branch stars)
PC=np.linspace(0.05,1.2,2,endpoint=True) 
dmagC=np.linspace(0.3,2,2,endpoint=True) 
JmKC=np.linspace(-0.01,0.15,2,endpoint=True)

typeD='Pulsator: GDor' # Non-radially pulsating high-order g-mode dwarf stars
PD=np.linspace(0.33,3,2,endpoint=True) 
dmagD=np.linspace(0,0.1,2,endpoint=True) 
JmKD=np.linspace(-0.01,0.3,2,endpoint=True)

typeE='Pulsator: Cepheid' # Radially pulsating, high luminosity (classes Ib-II) variables 
PE=np.linspace(1,140,2,endpoint=True) 
dmagE=np.linspace(0.03,2,endpoint=True) 
JmKE=np.linspace(0.15,0.8,2,endpoint=True)

typeJ='Pulsator: BXCIR'#Extreme helium stars --  pulsations driven by kappa mechanism through Z-bump instability. 
PJ=np.linspace(0.08,1.2,2,endpoint=True) 
dmagJ=np.linspace(0.08,0.2,2,endpoint=True) 
JmKJ=np.linspace(-99,0.15,2,endpoint=True)

typeK='Pulsator: CW' #Pulsating variables of the galactic spherical component (old disk) population
PK=np.linspace(0.8,35,2,endpoint=True) 
dmagK=np.linspace(0.3,1.2,2,endpoint=True) 
JmKK=np.linspace(0.15,0.5,2,endpoint=True)

##Rotators
typeF='RotVar'
PF=np.linspace(0.14,100,2,endpoint=True) 
dmagF=np.linspace(-99,99,2,endpoint=True) 
JmKF=np.linspace(0.3,99,2,endpoint=True)

typeG='Rot Var: ACV' #MS stars with spectral types B8p-A7p displaying strong B fields
PG=np.linspace(0.5,160,2,endpoint=True) 
dmagG=np.linspace(0.01,0.1,2,endpoint=True) 
JmKG=np.linspace(-0.17,0.15,2,endpoint=True)

typeH='Rot Var: BY'#Axial rotation of star with a variable degree of non-uniformity of surface brightness (spots) and chromospheric activity. 
PH=np.linspace(0.3,120,2,endpoint=True) 
dmagH=np.linspace(0.03,0.5,2,endpoint=True) 
JmKH=np.linspace(0.5,99,2,endpoint=True)

typeI='Rot Var: FKCOM'#Rapidly rotating single G and K-type giants with nonuniform surface brightnesses.
PI=np.linspace(0.5,25,2,endpoint=True) 
dmagI=np.linspace(0.01,0.08,2,endpoint=True) 
JmKI=np.linspace(0.3,0.8,2,endpoint=True)


Often not possible to disambiguate between rotational variables, ellipsoidal variables, and some classes of pulsational variables (e.g., gamma dors).  
Generally speaking, any G, K, M, or late F dwarf stars exhibiting continuous (i.e., not an eclipsing, flaring, or transiting planet light curve) quasi-periodic variability with periods between ~1 day and 100 days are probably a rotational variable. Those with periods less than ~1 day may either be rapid rotators or ellipsoidal variables (*ellipsoidal variables are sinusoidal*, so if it isn't sinusoidal that it is a good indicator that the source is a rotational variable, if it is, then you can't really tell if it's ellipsoidal or rotational).  For earlier type stars you are usually looking at some type of pulsation.

In [4]:
filesleft=glob.glob('/Users/msoares/Dropbox/C0_VariableCull5/Sources/**/*png*', recursive=True) #this just is used to read the names of all files
BestPeriods=Table.read("BestPeriods_Mags.dat",format='ascii') #this reads in the optimal periods and dmags determined from periodograms
spectralinfo=ascii.read('/Users/msoares/Dropbox/K20/Notebooks/Data_Files/Spectral_Cat.dat') #this reads in the J-mag and K-mag information

#I used ASTROBASE to discern the dmag values by fitting a Fourier Transform to the LC and determining the amplitude.

JminK_check=1 #mark as true if you want to use J-K color to classify

for i in range(0,1):#len(filesleft)):
    types=[] #variable classes from period check
    #separate out the name of the variable from your list of pngs 
    if ('HAT' in filesleft[i]):
        HATID=filesleft[i].split('HAT')[1]
        HATID='HAT'+HATID
    if ('UCAC' in filesleft[i]):
        HATID=filesleft[i].split('UCAC')[1]
        HATID='UCAC'+HATID
    HATID=HATID.split('.png')[0]
    if ('_B' in filesleft[i]):
        TrueHATID=HATID
        pathway='/Users/msoares/Desktop/C0_VariableCull5/**/'+str(TrueHATID)+'.png*'
        HATID=HATID.split('_B')[0] 
    else:
        TrueHATID=HATID
        pathway='/Users/msoares/Desktop/C0_VariableCull5/**/'+str(HATID)+'.png*'
    #get information for this particular star
    index = np.nonzero(BestPeriods['HATID']==HATID)[0][0]
    P0=(BestPeriods['PBest'][index])
    dmag=(BestPeriods['dmag'][index])
    index2 = np.nonzero(spectralinfo['HATID']==HATID)[0][0]
    JminK=spectralinfo['Jmag'][index2]-spectralinfo['Kmag'][index2]
    # if you want to apply a correction factor, add it here...you'll need to use the Gaia DR2 catalog and have a distance handy
    #corr=0.25
    #JminK=JminK-corr
    if JminK>O_c[0] and JminK<O_c[1]:
        SC='O'
    if JminK>O_c[1] and JminK<B_c[1]:
        SC='B'
    if JminK>B_c[1] and JminK<A_c[1]:
        SC='A'
    if JminK>A_c[1] and JminK<F_c[1]:
        SC='F'
    if JminK>F_c[1] and JminK<G_c[1]:
        SC='G'
    if JminK>G_c[1] and JminK<K_c[1]:
        SC='K'
    if JminK>K_c[1] and JminK<M_c[1]:
        SC='M'
    print(HATID,P0,dmag,JminK,SC)
    ################################################
    # Now that we know our source, let's classify! #
    ########### Classify based on Period ###########
    if (P0>min(PA)) and (P0 < max(PA)):
        types.append(typeA)
    if (P0>min(PB)) and (P0 < max(PB)):
        types.append(typeB)
    if (P0>min(PC)) and (P0 < max(PC)):
        types.append(typeC)
    if (P0>min(PD)) and (P0 < max(PD)):
        types.append(typeD)
    if (P0>min(PE)) and (P0 < max(PE)):
        types.append(typeE)
    if (P0>min(PF)) and (P0 < max(PF)):
        types.append(typeF)
    if (P0>min(PG)) and (P0 < max(PG)):
        types.append(typeG)
    if (P0>min(PH)) and (P0 < max(PH)):
        types.append(typeH)
    if (P0>min(PI)) and (P0 < max(PI)):
        types.append(typeI)
    if (P0>min(PJ)) and (P0 < max(PJ)):
        types.append(typeJ)
    if (P0>min(PK)) and (P0 < max(PK)):
        types.append(typeK)
    ############ Classify based on Mag #############
    types2=[] #variable classes from period and dmag check
    for b in types:
        if b==typeA:
            if (dmag>min(dmagA)) and (dmag < max(dmagA)):
                types2.append(typeA)
        if b==typeB:
            if (dmag>min(dmagB)) and (dmag < max(dmagB)):
                types2.append(typeB)
        if b==typeC:
            if (dmag>min(dmagC)) and (dmag < max(dmagC)):
                types.append(typeC)
        if b==typeD:
            if (dmag>min(dmagD)) and (dmag < max(dmagD)):
                types2.append(typeD)
        if b==typeE:
            if (dmag>min(dmagE)) and (dmag < max(dmagE)):
                types2.append(typeE)
        if b==typeF:
            if (dmag>min(dmagF)) and (dmag < max(dmagF)):
                types2.append(typeF)
        if b==typeG:
            if (dmag>min(dmagG)) and (dmag < max(dmagG)):
                types2.append(typeG)
        if b==typeH:
            if (dmag>min(dmagH)) and (dmag < max(dmagH)):
                types2.append(typeH)
        if b==typeI:
            if (dmag>min(dmagI)) and (dmag < max(dmagI)):
                types2.append(typeI)
        if b==typeJ:
            if (dmag>min(dmagJ)) and (dmag < max(dmagJ)):
                types2.append(typeJ)
        if b==typeK:
            if (dmag>min(dmagK)) and (dmag < max(dmagK)):
                types2.append(typeK)
        ###Next classify based on J-K###
        types3=[] #variable classes from period + dmag + J-K check
        if JminK_check==1:
            for k in types2:
                if k==typeA:
                    if (JminK>min(JmKA)) and (JminK < max(JmKA)):
                        types3.append(typeA)
                if k==typeB:
                    if (JminK>min(JmKB)) and (JminK < max(JmKB)):
                        types3.append(typeB)
                if k==typeC:
                    if (JminK>min(JmKC)) and (JminK < max(JmKC)):
                        types3.append(typeC)
                if k==typeD:
                    if (JminK>min(JmKD)) and (JminK < max(JmKD)):
                        types3.append(typeD)
                if k==typeE:
                    if (JminK>min(JmKE)) and (JminK < max(JmKE)):
                        types3.append(typeE)
                if k==typeF:
                    if (JminK>min(JmKF)) and (JminK < max(JmKF)):
                        types3.append(typeF)
                if k==typeG:
                    if (JminK>min(JmKG)) and (JminK < max(JmKG)):
                        types3.append(typeG)
                if k==typeH:
                    if (JminK>min(JmKH)) and (JminK < max(JmKH)):
                        types3.append(typeH)
                if k==typeI:
                    if (JminK>min(JmKI)) and (JminK < max(JmKI)):
                        types3.append(typeI)
                if k==typeJ:
                    if (JminK>min(JmKJ)) and (JminK < max(JmKJ)):
                        types3.append(typeJ)       
                if k==typeK:
                    if (JminK>min(JmKK)) and (JminK < max(JmKK)):
                        types3.append(typeK)       
    print('\n',types,'\n',types2,'\n',types3) #what were the variables discerned at each step?
print('\n Complete')



HAT-264-0001505 2.14118618445946 0.28084 0.44399999999999906 G

 ['Pulsator: GDor', 'Pulsator: Cepheid', 'RotVar', 'Rot Var: ACV', 'Rot Var: BY', 'Rot Var: FKCOM', 'Pulsator: CW'] 
 ['Pulsator: Cepheid', 'RotVar', 'Rot Var: BY'] 
 ['Pulsator: Cepheid', 'RotVar']

 Complete
